Скачайте вот этот текст -  https://github.com/mannefedov/compling_nlp_hse_course/blob/master/data/zhivago.txt

In [1]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Mounted at /content/gdrive


In [2]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks')

In [ ]:
!pip install pymorphy2
!pip install pymystem3==0.1.10
!pip install rusenttokenize
!pip install razdel
!pip install wget

In [ ]:
import wget

wget.download("https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/zhivago.txt", 'raw_text.txt')

'raw_text.txt'

**Задание 1 (2 балла).**

Отчистите текст от мусора (тэгов, хешей и тп) с помощью регулярных выражений. Постарайтесь убрать весь мусор, но если что-то удалить не получается, то не мучайтесь. Главное, чтобы мусор не проявлялся в результатах следующих заданий.

In [4]:
f = open('raw_text.txt', 'r', encoding = 'utf-8-sig')
text = f.read()
f.close()

In [5]:
text[:300]

'<author><first-name>Борис</first-name><middle-name>Леонидович</middle-name><last-name>Пастернак</last-name></author>\n<book-title>Доктор Живаго</book-title>\n<annotation><p>«Доктор Живаго» - итоговое произведение Бориса Пастернака, книга всей его жизни. Этот роман принес его автору мировую известность'

In [6]:
import re
reTag = re.compile('<.*?>', re.DOTALL) #очистим текст от тегов
reh = re.compile('[\\n|\\xa0]', re.DOTALL)
text_clean = re.sub(reTag, ' ', text) #заменим на пробелы, чтобы слова не склеились
text_clean = re.sub(reh, ' ', text_clean)
text_clean = re.sub('  +', ' ', text_clean) #уберем лишние пробелы

In [7]:
text_clean[:300]

' Борис Леонидович Пастернак Доктор Живаго «Доктор Живаго» - итоговое произведение Бориса Пастернака, книга всей его жизни. Этот роман принес его автору мировую известность и Нобелевскую премию, присуждение которой обернулось для поэта оголтелой политической травлей, обвинениями в «измене Родине» и в'

**Задание 2. (2 балла)**

Приведите очищенный текст к нижнему регистру, удалите все знаки пунктуации, разделите на предложения библиотекой rusenttokenize, токенизируйте библиотекой razdel_tokenize. 

Ответьте на следующие вопросы:

1) есть ли в тексте повторяющиеся корректные предложения? если да то какие? (если находится мусор то вернитесь к заданию 1 и постарайтесь избавиться от него)

2) какой самый частотный токен в тексте длиннее 6 символов?

Чтобы текст можно было разделить на предложения, уберем пунктуацию после деления, тогда же и приведем предложения к нижнему регистру

In [8]:
from rusenttokenize import ru_sent_tokenize

In [9]:
sents = ru_sent_tokenize(text_clean)

In [10]:
sents[:5]

['Борис Леонидович Пастернак Доктор Живаго «Доктор Живаго» - итоговое произведение Бориса Пастернака, книга всей его жизни.',
 'Этот роман принес его автору мировую известность и Нобелевскую премию, присуждение которой обернулось для поэта оголтелой политической травлей, обвинениями в «измене Родине» и в результате стоило ему жизни.',
 '«Доктор Живаго» - роман, сама ткань которого убедительнее свидетельствует о чуде, чем все размышления доктора и обобщения автора.',
 'Человек, который так пишет, бесконечно много пережил и передумал, и главные его чувства на свете - восхищенное умиление и слезное сострадание; конечно, есть в его мире место и презрению, и холодному отстранению - но не в них суть.',
 'Роман Пастернака - оплакивание прежних заблуждений и их жертв; те, кто не разделяет молитвенного восторга перед миром, достойны прежде всего жалости.']

In [11]:
#убираем всю пунктуцацию и приводим к нижнему регистру
sents_clean = [re.sub("[^\s\w]+", " ", i).lower() for i in sents]
sents_clean = [re.sub('  +', ' ', i) for i in sents_clean] #уберем лишние пробелы

In [12]:
sents_clean[:5]

['борис леонидович пастернак доктор живаго доктор живаго итоговое произведение бориса пастернака книга всей его жизни ',
 'этот роман принес его автору мировую известность и нобелевскую премию присуждение которой обернулось для поэта оголтелой политической травлей обвинениями в измене родине и в результате стоило ему жизни ',
 ' доктор живаго роман сама ткань которого убедительнее свидетельствует о чуде чем все размышления доктора и обобщения автора ',
 'человек который так пишет бесконечно много пережил и передумал и главные его чувства на свете восхищенное умиление и слезное сострадание конечно есть в его мире место и презрению и холодному отстранению но не в них суть ',
 'роман пастернака оплакивание прежних заблуждений и их жертв те кто не разделяет молитвенного восторга перед миром достойны прежде всего жалости ']

In [13]:
#при первой попытке найти повторяющиеся предложения
#в частотном списке оказывались одинаковые предложения по типу "да" и " да", которые отличались лишь наличием пробелов в начале строки
#регулярное выражение ниже убирает эти пробелы
respace = re.compile(r'^ +', flags=re.MULTILINE)
sents_clean = [re.sub(respace, '', i) if i.startswith(' ') == True else i for i in sents_clean]

In [14]:
import collections
c = collections.Counter()
for sent in sents_clean:
  c[sent] += 1

In [15]:
from collections import OrderedDict

sent_dict = OrderedDict(sorted(dict(c).items(), key=lambda t: t[1], reverse = True))

In [19]:
k=0
for i in sent_dict.keys():
  print(i, sent_dict[i])
  k+=1
  if k==20:
    break

да  21
не правда ли  10
нет  10
хорошо  8
а  7
ну как же  6
знаю  5
погоди  5
спасибо  5
я знаю  5
конечно  4
маркел  4
разумеется  4
запамятовал  4
еще бы  4
сеялки  4
молотилки  4
неужели правда  4
ха ха ха  3
куда там  3


Повторяющие предложения есть. Можно отметить, что это по-видимому различные короткие реплики персонажей

In [20]:
from razdel import sentenize
from razdel import tokenize as razdel_tokenize

In [21]:
all_tokens = []
for sent in sents_clean:
  tokens = [token.text for token in list(razdel_tokenize(sent))]
  for token in tokens:
    all_tokens.append(token)

In [22]:
w = collections.Counter()
for token in all_tokens:
  w[token] += 1

In [23]:
token_dict = OrderedDict(sorted(dict(w).items(), key=lambda t: t[1], reverse = True))

In [24]:
for i in token_dict.keys():
  if len(i) > 6:
    print(i, token_dict[i])
    break

андреевич 289


Получилось, что самый частотный токен длиннее 6 символов - это "Андреевич".

**Задание 3. (2 балла)**

Сделайте стемминг и найдите по несколько частотных примеров на каждый из видов ошибок:

1) два разных слова ошибочно свелись к одинаковой основе

2) слово не изменилось после стемминга (слово должно быть русским и длиннее 4 символов)

(не делайте это задание полностью вручную, напишите правила, которые отловят потенциальные ошибки и выберите из них)

In [25]:
from nltk.stem.snowball import SnowballStemmer

In [26]:
stemmer = SnowballStemmer('russian')

In [27]:
stem_dict = {}
for word in all_tokens:
  if stemmer.stem(word) not in stem_dict.keys():
    stem_dict[stemmer.stem(word)] = [word]
  else:
    stem_dict[stemmer.stem(word)].append(word)

In [28]:
stem_dict_2 = OrderedDict(sorted(stem_dict.items(), key=lambda t: len(t[1]), reverse = True))

In [40]:
k=0
for i in stem_dict_2.keys():
  if 2 < len(i) < 4:
    for j in range(1,len(stem_dict_2[i])):
      if stem_dict_2[i][j] != stem_dict_2[i][0]:
        print(i, stem_dict_2[i][:10])
        break
    k+=1
    if k==30:
      break

как ['как', 'как', 'какие', 'как', 'как', 'как', 'как', 'как', 'как', 'как']
что ['что', 'что', 'что', 'что', 'что', 'что', 'что', 'что', 'что', 'что']
был ['был', 'было', 'был', 'был', 'был', 'были', 'был', 'были', 'было', 'был']
так ['так', 'так', 'таков', 'так', 'таким', 'так', 'такой', 'так', 'такое', 'таких']
все ['всей', 'все', 'все', 'все', 'все', 'всею', 'все', 'всей', 'всей', 'всей']
сво ['своей', 'своего', 'свою', 'свое', 'своих', 'своих', 'своей', 'своих', 'своей', 'своей']
сам ['сама', 'самое', 'самому', 'самое', 'самого', 'самого', 'самое', 'самих', 'самого', 'самого']
одн ['одном', 'одно', 'одном', 'одной', 'одна', 'одно', 'одна', 'одном', 'одного', 'одни']
над ['надо', 'надо', 'надо', 'надо', 'над', 'над', 'над', 'надо', 'надо', 'надо']
дом ['дома', 'домов', 'дома', 'домой', 'домом', 'доме', 'дома', 'дома', 'дома', 'дому']
себ ['себя', 'себя', 'себе', 'себе', 'себя', 'себе', 'себя', 'себе', 'себе', 'себе']
под ['под', 'под', 'под', 'под', 'под', 'под', 'под', 'под', 'под

Были найдены такие случаи приведения разных слов к одной основе: как - какие, вид - видеть, над - надо, меня - менее

Однако в подобных случаях сведение к одной основе, кажется, вполне оправдано, видимо для выявления именно вариантов с неправильным стеммингом, нужна какая-то дополнительная обработка

In [43]:
no_change = {}
for word in all_tokens:
  if len(stemmer.stem(word)) > 4 and word == stemmer.stem(word):
    if word not in no_change.keys():
      no_change[word] = stemmer.stem(word)

In [45]:
k=0
for i in no_change.keys():
  print(i, no_change[i])
  k+=1
  if k==10:
    break

борис борис
леонидович леонидович
пастернак пастернак
доктор доктор
принес принес
человек человек
пишет пишет
жертв жертв
перед перед
строк строк


Можно заметить, что слово "пишет" не изменилось, хотя основа должна была бы быть "пиш"

**Задание 4 (1 балл).**

Проанализируйте список стоп-слов из нлтк (для русского). Какие ещё слова вы бы туда добавили? (5 слов будет достаточно, не забудьте аргументировать ваш выбор)


In [47]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [54]:
print(stopwords.words("russian"))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

Мне кажется, стоило бы добавить к списку некоторые синонимы тех слов, что уже есть в списке, а также слова-паразиты (если конечно в исследовании именно они и не рассматриваются)

In [61]:
'почти' in stopwords.words("russian"), 'едва' in stopwords.words("russian")

(True, False)

In [60]:
'короче' in stopwords.words("russian"), 'таки' in stopwords.words("russian")

(False, False)

**Задание 5 (3 балла).**

Предобработайте текст двумя способами:

1) лемматизируйте токены с помощью pymorphy2

2) лемматизируйте текст с помощью mystem3 

Ответьте на вопрос:
Что в данном случае лучше для лемматизации mystem или pymorphy?

In [62]:
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem

In [63]:
morph = MorphAnalyzer()
mystem = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz


In [64]:
lemmas_pm2 = [morph.parse(token)[0].normal_form for token in all_tokens]

In [65]:
lemmas_mt3 = mystem.lemmatize(text_clean)

Сравним результаты лемматизации

In [71]:
text_clean[:400]

' Борис Леонидович Пастернак Доктор Живаго «Доктор Живаго» - итоговое произведение Бориса Пастернака, книга всей его жизни. Этот роман принес его автору мировую известность и Нобелевскую премию, присуждение которой обернулось для поэта оголтелой политической травлей, обвинениями в «измене Родине» и в результате стоило ему жизни. «Доктор Живаго» - роман, сама ткань которого убедительнее свидетельств'

In [79]:
print(lemmas_pm2[:51])

['борис', 'леонидович', 'пастернак', 'доктор', 'живаго', 'доктор', 'живаго', 'итоговый', 'произведение', 'борис', 'пастернак', 'книга', 'весь', 'он', 'жизнь', 'этот', 'роман', 'принести', 'он', 'автор', 'мировой', 'известность', 'и', 'нобелевский', 'премия', 'присуждение', 'который', 'обернуться', 'для', 'поэт', 'оголтелый', 'политический', 'травля', 'обвинение', 'в', 'измена', 'родина', 'и', 'в', 'результат', 'стоить', 'он', 'жизнь', 'доктор', 'живаго', 'роман', 'сам', 'ткань', 'который', 'убедительный', 'свидетельствовать']


In [78]:
print(lemmas_mt3[:200])

[' ', 'борис', ' ', 'леонидович', ' ', 'пастернак', ' ', 'доктор', ' ', 'живаго', ' «', 'доктор', ' ', 'живаго', '» - ', 'итоговый', ' ', 'произведение', ' ', 'борис', ' ', 'пастернак', ', ', 'книга', ' ', 'весь', ' ', 'его', ' ', 'жизнь', '. ', 'этот', ' ', 'роман', ' ', 'приносить', ' ', 'он', ' ', 'автор', ' ', 'мировой', ' ', 'известность', ' ', 'и', ' ', 'нобелевский', ' ', 'премия', ', ', 'присуждение', ' ', 'который', ' ', 'обертываться', ' ', 'для', ' ', 'поэт', ' ', 'оголтелый', ' ', 'политический', ' ', 'травля', ', ', 'обвинение', ' ', 'в', ' «', 'измена', ' ', 'родина', '» ', 'и', ' ', 'в', ' ', 'результат', ' ', 'стоить', ' ', 'он', ' ', 'жизнь', '. ', '«', 'доктор', ' ', 'живаго', '» - ', 'роман', ', ', 'сам', ' ', 'ткань', ' ', 'который', ' ', 'убедительно', ' ', 'свидетельствовать', ' ', 'о', ' ', 'чудо', ', ', 'чем', ' ', 'весь', ' ', 'размышление', ' ', 'доктор', ' ', 'и', ' ', 'обобщение', ' ', 'автор', '. ', 'человек', ', ', 'который', ' ', 'так', ' ', 'писать', ', 

In [84]:
lemmas_mt3_clean = []
for i in lemmas_mt3:
  if re.findall(r'\w+', i) != []:
    lemmas_mt3_clean.append(i)

In [85]:
len(lemmas_pm2), len(lemmas_mt3), len(lemmas_mt3_clean)

(161800, 310221, 154149)

In [93]:
for i in range(150):
  if lemmas_pm2[i] != lemmas_mt3_clean[i]:
    print(i, all_tokens[i], lemmas_pm2[i], lemmas_mt3_clean[i])

13 его он его
17 принес принести приносить
27 обернулось обернуться обертываться
49 убедительнее убедительный убедительно
54 все всё весь
66 пережил пережить переживать
68 передумал передумать передумывать
71 его он его
75 восхищенное восхитить восхищенный
78 слезное слёзный слезный
81 есть есть быть
83 его он его
95 суть быть суть
102 их они их
114 всего весь все
119 стоит стоить стоять
132 этого это этот


Проверим контексты некоторых употреблений

In [95]:
for i in range(110, 120):
  print(all_tokens[i])

перед
миром
достойны
прежде
всего
жалости
перечитывать
доктора
живаго
стоит


In [97]:
for i in range(50, 60):
  print(all_tokens[i])

свидетельствует
о
чуде
чем
все
размышления
доктора
и
обобщения
автора


In [96]:
for i in range(130, 135):
  print(all_tokens[i])

строк
из
этого
романа
могут


In [99]:
for i in range(75, 100):
  print(all_tokens[i])

восхищенное
умиление
и
слезное
сострадание
конечно
есть
в
его
мире
место
и
презрению
и
холодному
отстранению
но
не
в
них
суть
роман
пастернака
оплакивание
прежних


Кажется, что расхождения в лемматизации возникают в основном с местоимениями и глаголами.

С одной стороны, Mystem не справляется с формами личных местоимений (его, их, всего), с другой - он лучше детектирует формы в контексте (этого, все), например, "этого романа" -> "этот", а не "это".

Также Pymorphy в отличие от Mystem сохраняет вид глагола (принести, пережить, передумать (ср. передумывать)), однако Mystem опять же лучше понимает значение в контексте (суть - быть - суть, есть - есть - быть)

Можно сделать вывод, что них обоих есть свои плюсы и минусы.